In [1]:
import glob
import os
import math
import json
import random
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator
import matplotlib.pyplot as plt
from PIL import ImageDraw


%matplotlib inline

# Fir virtualenv autocompletion.
%config Completer.use_jedi = False

DATA_ROOT = '../data/'

In [2]:
def load_mrxs_files(data_dir):
    full_paths = glob.glob(data_dir + '*.mrxs')
    return [os.path.basename(file) for file in full_paths]

In [3]:
def load_annotations(mrxs_files):
    anns = []
    ann_files = [ file.replace('mrxs', 'json') for file in mrxs_files]
    for file in ann_files:
        with open(DATA_ROOT + file) as f:
            ann_json = json.load(f)
            anns.append(ann_json)
    return tuple(anns)

In [4]:
mrxs_files = load_mrxs_files(DATA_ROOT)
annotations = load_annotations(mrxs_files)
slides = []
for file in mrxs_files:
    slides.append(OpenSlide(DATA_ROOT + file))

## Slide object parameters

In [5]:
slide = slides[0]
print("Levels: {}".format(slide.level_count))
for idx, dimensions in enumerate(slide.level_dimensions):
    print("\tLVL {} - {:8d} X {} px".format(idx, dimensions[0], dimensions[1]))

Levels: 10
	LVL 0 -   177152 X 416768 px
	LVL 1 -    88576 X 208384 px
	LVL 2 -    44288 X 104192 px
	LVL 3 -    22144 X 52096 px
	LVL 4 -    11072 X 26048 px
	LVL 5 -     5536 X 13024 px
	LVL 6 -     2768 X 6512 px
	LVL 7 -     1384 X 3256 px
	LVL 8 -      692 X 1628 px
	LVL 9 -      346 X 814 px


## Annotations file example 

Selecting an annotation and displaying the annotated section in the middle of image.

In [6]:
annotations[0]['annotations']['1']

{'geometry': {'annotation_type': 'ELLIPSE',
  'origin_point': [0, 0],
  'points': [[123620, 194814], [123722, 194887]]},
 'id': '1',
 'label': 'annotation_1',
 'tree_view_config': {'display_pattern': '{label}',
  'decoration_attr': 'figure_graphics_view_config.color',
  'decoration_size': None},
 'text_graphics_view_config': {'display_pattern': '{label}\\n{stats[text]}\\n{filter_results[text]}',
  'hidden': False,
  'background_color': '#32cd32'},
 'figure_graphics_view_config': {'hidden': False, 'color': '#32cd32'},
 'stats': {'text': 'area: 88µ²',
  'area': 88,
  'area_px': 5986,
  'area_text': 'area: 88µ²',
  'length': None,
  'length_px': None,
  'length_text': None},
 'filter_id': None,
 'filter_level': None,
 'filter_results': None,
 'user_attrs': {'z_index': 'type 1', 'ROI': 'true', 'label_color 1': '1'}}

In [27]:
def show_annotated_section(annotation, slide, image_size=512, save=False, filename=None):
    """
        Shows a single annotation centred in the image.
        image_size - returned image size
    """
    ann_type = annotation['geometry']['annotation_type']

    if ann_type != 'ELLIPSE':
        raise NotImplementedError
        
    colors = {
        'label_color 1' : 'blue',
        'label_color 2' : 'green',
        'label_color 3' : 'red',
        'label_color 4' : 'yellow'
    }
       
    
    # Annotations are for ellipses that are defined by
    # two points of bounding boxes.
    ann_points = annotation['geometry']['points']
    x1, y1 = ann_points[0][0], ann_points[0][1]
    x2, y2 = ann_points[1][0], ann_points[1][1]
    
    # Locate the region so that the annotation first
    # coordinate is in the middle of the section.
    img = slide.read_region(
        location = (x1 - image_size//2, y1 - image_size//2),
        level = 0,
        size = (image_size, image_size)
    )
    

    # Ellipse starts from the middle of image.
    point1 = (image_size//2, image_size//2)
    # Ends where the second box is relative to the image.
    point2 = (image_size//2 + (x2 -x1), image_size//2 + (y2 -y1))
    # Pillow draws only if point1 < point2,
    # so that the point1 is upper left and point2 is lower right.
    if point1 > point2:
        #Swap
        point1, point2 = point2, point1
        
    # Color according to label.
    # White to check if there are missing labels.
    outline_color = 'white' 
    for key in annotation['user_attrs'].keys():
        if key in colors.keys():
            outline_color = colors[key]
 
    # Draw an ellipse over the image
    draw = ImageDraw.Draw(img)
    draw.ellipse(
        [
            point1,
            point2
        ],
        width=5,
        outline=outline_color
    )
    
    if save:
        if filename == None:
            filename = annotation['label'] + '.png'
        img.save('./samples/' + filename)
    else:
          display(img)

In [36]:
show_annotated_section(annotations[0]['annotations']['70'], slide, 512, save=True)

## Creating list of slides

Slicing trough the image with a predefined resolution to create slides.
Collecting annotation information for each slide.

In [9]:
def create_tiles_with_annotation(annotations, slide, tile_size=1024):
    x_dim = slide.level_dimensions[0][0]
    y_dim = slide.level_dimensions[0][1]
    
    tile_annotations = []
    # Moving through image with tiles.
    for x in range(0, x_dim, 1024):
        for y in range(0, y_dim, 1024):
            tile_info = {}
            
            # Top left pixel of tile.
            x_0 = (x, y)
            # Bottom right pixel.
            x_1 = (x + tile_size, y + tile_size)

            # Collect all annotations inside tile.
            tile_anns = []
            # Check if there are annotations inside tile.
            for ann in annotations['annotations']:
                centres = annotations['annotations'][ann]['geometry']['points']

                ann_in_image = False
                for point in centres:
                    x_p, y_p = point
                    # Point inside tile.
                    if x_p < x_1[0] and x_p > x_0[0] and y_p < x_1[1] and y_p > x_0[1]:
                        ann_in_image = True

                if ann_in_image:
                    tile_anns.append(annotations['annotations'][ann])

            
            tile_info['top_left'] = x_0
            tile_info['image_size'] = tile_size
            tile_info['annotations'] = tile_anns
            
            tile_annotations.append(tile_info)
    return tile_annotations

# Statistics

Gather statistics about created slides

In [10]:
def show_tiles_statistics(annotated_tiles):
    tot_count = len(annotated_tiles)
    empty_images = 0
    annotated_images = 0
    max_annotations_in_image = 0
    
    for tile in annotated_tiles:
        if len(tile['annotations']) == 0:
            empty_images += 1
        else:
            annotated_images += 1
            num_annotations = len(tile['annotations'])
            if num_annotations > max_annotations_in_image:
                max_annotations_in_image = num_annotations

    print('Total slides: \t\t{}'.format(tot_count))
    print('Slides not annotated: \t{}'.format(empty_images))
    print("Annotated slides: \t{}".format(annotated_images))
    print("Max annotations: \t{}".format(max_annotations_in_image))


In [11]:
slide_annotated_tiles = create_tiles_with_annotation(annotations[0], slide)
show_tiles_statistics(slide_annotated_tiles)

Total slides: 		70411
Slides not annotated: 	70353
Annotated slides: 	58
Max annotations: 	19


In [38]:
def show_tile_annotations(annotated_tiles, slide, num_images=1, save=False, filename = None):
    """
    Display tiles with annotatied images
    """
    tiles_with_annotations = []

    # Select tiles only with annotations.
    for tile in annotated_tiles:
        if len(tile['annotations']) > 0:
            tiles_with_annotations.append(tile)

    # Add different color for different label.
    # The only key that changes in annotations file is
    # label_color_* that may represent label.
    colors = {
        'label_color 1' : 'blue',
        'label_color 2' : 'green',
        'label_color 3' : 'red',
        'label_color 4' : 'yellow'
    }

    sample = random.sample(tiles_with_annotations, num_images)
    for tile in sample:
        top_left = tile['top_left']

        img = slide.read_region(
            location = (
                top_left[0],
                top_left[1]
            ),
            level = 0,
            size = (tile['image_size'], tile['image_size'])
        )

        for ann in tile['annotations']:
            ann_points = ann['geometry']['points']
            x1, y1 = ann_points[0][0], ann_points[0][1]
            x2, y2 = ann_points[1][0], ann_points[1][1]
            # Drawing points are relative to the top left
            # as the new image top left is (0,0)
            point1 = (
                x1 - top_left[0], 
                y1 - top_left[1]
            )
            point2 = (
                x2 - top_left[0],
                y2 - top_left[1]
            )
            # Pillow draws only if point1 < point2,
            # so that the point1 is upper left and point2 is lower right.
            if point1 > point2:
                #Swap
                point1, point2 = point2, point1
                
            # Color according to label.
            outline_color = 'white'
            for key in ann['user_attrs'].keys():
                if key in colors.keys():
                    outline_color = colors[key]

            # Draw an ellipse over the image
            draw = ImageDraw.Draw(img)
            draw.ellipse(
                [
                    point1,
                    point2
                ],
                width=4,
                outline=outline_color
            )
        
        if save:
            if filename == None:
                filename = "tile_{}_{}.png".format(top_left[0], top_left[1])
            img.save('./samples/' + filename)
        else:
              display(img)

In [39]:
show_tile_annotations(slide_annotated_tiles, slide, num_images=3, save=True)